In [1]:
import numpy as np
from scipy.linalg import norm 
import igl
import meshplot as mp
import triangle as tr
from scipy.sparse.linalg import spsolve
import ipywidgets as iw
import time
from scipy.spatial.transform import Rotation

<h1 style="text-align:center; color:blue;"> Harmonic Coordinates for Character Articulation </h1>
<p style="text-align:center; color:black;"><strong>Author:</strong> Johan Moreira</p>
<p style="text-align:center; color:black;"><strong>Date:</strong> 05/21/2021</p>

This project is based on the paper Harmonic Coordinates for Character Articulation by Joshi, et al. In this notebook I consider the problem of creating and controlling planar figure deformations used to articulate characters for use in high-end applications. My goal is to show that harmonic coordinates improves upon existing planar figure deformation techniques. 

The deformations performed here are controlled using a topologically flexible structure, called a cage, that consists of a closed two dimensional mesh. I will show that harmonic coordinates are generalized barycentric coordinates that can be extended to any dimension. Moreover, harmonic coordinates are the first system of generalized barycentric coordinates that are non-negative even in strongly concave situations, and their magnitude falls off with distance as measured within the cage.

<img src="data/fig1.png">

The steps to accomplish the deformations are the following:
<ol>
  <li>Reading the Data</li>
  <li>Creating the Cage Manually</li>
  <li>Triangulation of the Cage</li>
  <li>Obtain Harmonic Coordinates by Solving Laplace's Equation</li>
  <li>Finding intersection between Mesh Points and Cage Triangles</li>
  <li>6.- Deforming Cage Vertex Deforming Mesh by Using Harmonic Coordinates and Barycentric Coordinates</li>
</ol>

<h3 style="color: green">1.- Reading Data</h3>

Note that the data contains the mesh that will be deformed and a selector mesh that will be used to create the cage

In [6]:
# Loading Mesh
v, f = igl.read_triangle_mesh("data/woody-lo.off")
v /= 10

# Loading Selector
s_v, s_f, s_c = igl.read_off("data/selector.off")
s_v *= 100

# Min Max Values for mesh
min_x = v[:,0].min()
min_y = v[:,1].min()
max_x = v[:,0].max()
max_y = v[:,1].max()

<h3 style="color: green">2.- Creating the Cage</h3>

To create the cage, use the slider below and click on the Add Vertex Button to add all the points that will compose the cage. Once all the points are defined, then click on the draw Cage button to create the cage

In [8]:
# Cage Vertices
cage_vertices = []
edges = []

button = iw.Button(description="Add Vertex!")
draw_cage = iw.Button(description="Draw Cage!")

# Set Callback
def add_vertex(b):
    cage_vertices.append(list(sf.coord[0,0:2]))
    if(len(cage_vertices) >= 1):
        paint_ui.add_points(np.array(cage_vertices),shading={"point_color": "green", "point_size": 10})

    
def add_edges(b):
    if len(cage_vertices) > 1:
        for i in range(len(cage_vertices)-1):
            edges.append([i,i+1])
        edges.append([len(cage_vertices)-1,0])
        paint_ui.add_edges(np.array(cage_vertices),np.array(edges))
    else:
        print("Need at least 2 vertices to draw a cage")
    
button.on_click(add_vertex)
draw_cage.on_click(add_edges)

# Meshplot
paint_ui = mp.plot(v,f,shading={"wireframe": True,"width": 400, "height": 300})
paint_ui.add_mesh(s_v,s_f,c=s_c)

# Display Buttons
display(iw.HBox([button, draw_cage]))

def sf(x,y):
    s_v_1 = s_v.copy()
    s_v_1 += np.array([x,y,0])
    paint_ui.update_object(oid=1, vertices=s_v_1, colors=s_c)
    sf.coord = s_v_1

mp.interact(sf, 
            x = iw.FloatSlider(min=min_x-5, max=max_x+5, value=0.0, step=0.01),
            y = iw.FloatSlider(min=min_y-5, max=max_y+5, value=0.0, step=0.01))

"""Note: This implementation works only for convex cages. If need to redraw the cage, please re-run this kernel"""

Renderer(camera=PerspectiveCamera(aspect=1.3333333333333333, children=(DirectionalLight(color='white', intensi…

interactive(children=(FloatSlider(value=0.0, description='x', max=39.5829, min=-4.866076, step=0.01), FloatSli…

'Note: This implementation works only for convex cages. If need to redraw the cage, please re-run this kernel'

In [9]:
# Saving the cage vertices and edges
filename_1 = 'cage_vertices'
filename_2 = 'edge_cage'
np.save('data/'+filename_1, np.array(cage_vertices))
np.save('data/'+filename_2, np.array(edges))

<h3 style="color: green">3.- Triangulating and Plotting Cage</h3>

In [10]:
# Predefined cage. Use these files if you do not want to create your own cage
f1 = 'cage_vertices_predifined'
f2 = 'cage_edges_predifined'

# Use filename_1 and filename_2 for your own defined cage

#Reading the cage data
#c_vertices = np.load('data/'+f1+'.npy')
#c_edges = np.load('data/'+f2+'.npy')
c_vertices = np.load('data/'+filename_1+'.npy')
c_edges = np.load('data/'+filename_2+'.npy')
n = c_vertices.shape[0]

A = dict(vertices=c_vertices)
B = tr.triangulate(A,'qa5')
v_cage = B['vertices']
f_cage = B['triangles']
colors_cage = np.ones([v_cage.shape[0],3])*0.4


In [11]:
q = mp.plot(v_cage, f_cage, c =colors_cage, shading={"wireframe": True, "width": 400, "height": 300})
q.add_mesh(v, f)
q.add_points(v_cage, shading={"point_color": "green", "point_size": 5})

Renderer(camera=PerspectiveCamera(aspect=1.3333333333333333, children=(DirectionalLight(color='white', intensi…

2

<h3 style="color: green">4.- Obtaining Harmonic Coordinates</h3>

In [12]:
# Function that will define the constraints
def get_contraint(k):
    z = np.zeros(v_cage.shape[0])
    z[k] = 1
    if k == 0:
        neig1 = k+1
        neig2 = v_b[n-1]
    elif k == n-1:
        neig1 = 0
        neig2 = k-1
    else:
        neig1 = k+1
        neig2 = k-1
        
    d1 = norm(v_cage[k,:]-v_cage[neig1,:])
    d2 = norm(v_cage[k,:]-v_cage[neig2,:])
    
    for j in range(v_b.shape[0]):
        if k != j and j not in list(range(n)):
            dist1 = norm(v_cage[neig1,:]-v_cage[v_b[j],:])
            dist2 = norm(v_cage[k,:]-v_cage[v_b[j],:])
            dist3 = norm(v_cage[neig2,:]-v_cage[v_b[j],:])
            
            if dist1 < d1 and dist2 < d1:
                z[v_b[j]] = dist1/d1
            if dist3 < d2 and dist2 < d2:
                z[v_b[j]] = dist3/d2
    return z


# Find boundary vertices
e = igl.boundary_facets(f_cage)
v_b = np.unique(e)

## List of all vertex indices
v_all = np.arange(v_cage.shape[0])

## List of interior indices
v_in = np.setdiff1d(v_all, v_b)

## Construct and slice up Laplacian
l = igl.cotmatrix(v_cage, f_cage)
l_ii = l[v_in, :][:, v_in]
l_ib = l[v_in, :][:, v_b]

## List that will hold the harmonic coordinates
harmonic_coordinates = np.zeros([n,v_cage.shape[0]])

for i in range(v_b.shape[0]):
    if i in list(range(n)):
        z = get_contraint(i)
        bc = z[v_b]
        z[v_in] = spsolve(-l_ii, l_ib.dot(bc))
        harmonic_coordinates[i,:]= z    
        

<h3 style="color: green">5.- Finding intersection between Mesh Points and Cage Triangles</h3>

In [13]:
def find_barycentric_coordinate_tri_2d(p):
    barycentric_coordinates = []
    face_id = -1
    for k in range(f_cage.shape[0]):
        Ax = v_cage[f_cage[k,0],0] - v_cage[f_cage[k,2],0] 
        Ay = v_cage[f_cage[k,0],1] - v_cage[f_cage[k,2],1]
        Bx = v_cage[f_cage[k,1],0] - v_cage[f_cage[k,2],0] 
        By = v_cage[f_cage[k,1],1] - v_cage[f_cage[k,2],1]
        Cx = p[0] - v_cage[f_cage[k,2],0]
        Cy = p[1] - v_cage[f_cage[k,2],1]
        
        beta = (Ay*Cx - Ax*Cy)/(Ay*Bx-Ax*By)
        alpha = (By*Cx - Bx*Cy)/(By*Ax-Bx*Ay)
        
        if (alpha <= 1 and alpha >= 0) and (beta <= 1 and beta >= 0) and (alpha+beta<1):
            face_id = k
            barycentric_coordinates.append([alpha, beta, 1-alpha-beta])
        
    return [face_id, barycentric_coordinates]

# Arrays that will store the barycentric coordinates of each mesh vertex
# with respect to the intersection triangle, and id of intersecting triangle
barycentric_coordinates = []
intersection_id = []

for vert in v:
    ans = find_barycentric_coordinate_tri_2d(vert)
    barycentric_coordinates.append(ans[1][0])
    intersection_id.append(ans[0])
barycentric_coordinates = np.array(barycentric_coordinates)

<h3 style="color: green">6.- Deforming Cage Vertex Deforming Mesh by Using Harmonic Coordinates and Barycentric Coordinates</h3>

In [14]:
v_cage_2 = np.hstack((v_cage,np.zeros([v_cage.shape[0],1])))
v_modified = v.copy()
handle_vertices = v_cage_2.copy()
pos_f_saver = np.zeros((n, 3))

def pos_f(s,x,y):
    handle_vertices[int(s)] = v_cage_2[int(s)] + np.array([[x,y,0]])
    pos_f_saver[int(s) - 1] = [x,y,0]
    t0 = time.time()
    v_deformed_cage = pos_deformer_cage(handle_vertices)
    p.update_object(oid=0,vertices = v_deformed_cage)
    v_deformed_mesh = pos_deformer_mesh(v_deformed_cage)
    p.update_object(oid=1,vertices = v_deformed_mesh)
    t1 = time.time()
    print('FPS', 1/(t1 - t0))

def pos_deformer_cage(vertices):
    for vi in v_in:
        total = np.zeros(3)
        for vc in range(n):
            total += vertices[vc,:] * harmonic_coordinates[vc,vi]
        vertices[vi] = total
        
    for vb in np.setdiff1d(v_b,list(range(n))):
        total2 = np.zeros(3)
        for vc in range(n):
            total2 += vertices[vc,:] * harmonic_coordinates[vc,vb]
        vertices[vb] = total2
        
    return vertices

def pos_deformer_mesh(vertices):
    for idx, value in enumerate(intersection_id):
        v_modified[idx,:] = (vertices[f_cage[value,0],:] * barycentric_coordinates[idx,0] 
                           + vertices[f_cage[value,1],:] * barycentric_coordinates[idx,1] 
                           + vertices[f_cage[value,2],:] * barycentric_coordinates[idx,2])  
    return v_modified
        
        

def widgets_wrapper():
    segment_widget = iw.Dropdown(options=list(range(n)))
    translate_widget = {i:iw.FloatSlider(min=-5, max=5, value=0, step=0.01) 
                        for i in 'xy'}

    def update_seg(*args):
        (translate_widget['x'].value,translate_widget['y'].value,
        _) = pos_f_saver[segment_widget.value]
    segment_widget.observe(update_seg, 'value')
    widgets_dict = dict(s=segment_widget)
    widgets_dict.update(translate_widget)
    return widgets_dict

In [17]:
## Widget UI
p = mp.plot(handle_vertices, f_cage, c=colors_cage, shading={"width": 400, "height": 300})
p.add_mesh(v,f)
"""The cage is shown in grey"""
iw.interact(pos_f,
            **widgets_wrapper())

Renderer(camera=PerspectiveCamera(aspect=1.3333333333333333, children=(DirectionalLight(color='white', intensi…

interactive(children=(Dropdown(description='s', options=(0, 1, 2, 3, 4, 5), value=0), FloatSlider(value=0.0, d…

<function __main__.pos_f(s, x, y)>